# Title of notebook

Brief 1-2 sentence description of notebook.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Imports of all used packages and libraries
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import seaborn as sns


In [3]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

## Inputs & Data

Explanation of each input and where it comes from.

In [4]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [25]:
CHANNEL_MAPPING_DF = pd.read_excel("../../data/channel_mapping.xlsx")
TONE_TIMESTAMP_DF = pd.read_excel("../../data/rce_tone_timestamp.xlsx", index_col=0)
channel_map_and_all_trials_df = pd.read_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")
merged_trials_and_video = pd.read_pickle("./proc/trial_SLEAP_and_metadata.pkl")

In [6]:
TIME_HALFBANDWIDTH_PRODUCT = 2
TIME_WINDOW_DURATION = 1
TIME_WINDOW_STEP = 0.5
RESAMPLE_RATE=1000


In [7]:
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}

In [8]:
BASELINE_OUTCOME_TO_COLOR = {'lose': "red",
 'omission': "orange",
 'rewarded': "green",
 'win': "blue"}

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [9]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [33]:
merged_trials_and_video.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,video_name,...,baseline_thorax_velocity,trial_thorax_velocity,trial_chunked_thorax_velocity,baseline_chunked_thorax_velocity,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,binned_baseline_thorax_velocity,binned_trial_thorax_velocity
0,30950500,20221214_125409_om_and_comp_6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,27998597,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,28574,1.0,6_1_top_1_base_2_vs_6_3,win_non_comp,20221214_125409_om_and_comp_6_1_and_6_3.1,...,"[1.2476140081638165, 1.1366289071218907, 1.068...","[0.18170472850681613, 0.16961378983662276, 0.2...","[0.49872187702582166, 1.0783705041183183, 1.43...","[1.6838472335457935, 1.3726890573782602, 1.887...","[(27998597, 28018597), (28018597, 28038597), (...","[(27798597, 27818597), (27818597, 27838597), (...","[(1399929, 1400929), (1400929, 1401929), (1401...","[(1389929, 1390929), (1390929, 1391929), (1391...","[0, 0, 0, 0, 1, 1, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,32550496,20221214_125409_om_and_comp_6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,29598593,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,30569,1.0,6_1_top_1_base_2_vs_6_3,win_non_comp,20221214_125409_om_and_comp_6_1_and_6_3.1,...,"[2.7236727895073223, 2.628037603947678, 2.5298...","[0.38577700470932025, 0.16352840888997497, 0.0...","[0.3129891797005337, 0.210547774617734, 0.2517...","[1.458421075097482, 0.23609874826191488, 0.073...","[(29598593, 29618593), (29618593, 29638593), (...","[(29398593, 29418593), (29418593, 29438593), (...","[(1479929, 1480929), (1480929, 1481929), (1481...","[(1469929, 1470929), (1470929, 1471929), (1471...","[0, 0, 0, 0, 0, 1, 1, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,34350494,20221214_125409_om_and_comp_6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,31398591,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,32813,1.0,6_1_top_1_base_2_vs_6_3,win_non_comp,20221214_125409_om_and_comp_6_1_and_6_3.1,...,"[5.210405290871598, 5.31421978904958, 5.379251...","[0.16560567036540477, 0.1278165569746918, 0.09...","[0.19868857983610794, 0.48118846206642074, 0.4...","[3.439626522203859, 1.1535882532998658, 0.0565...","[(31398591, 31418591), (31418591, 31438591), (...","[(31198591, 31218591), (31218591, 31238591), (...","[(1569929, 1570929), (1570929, 1571929), (1571...","[(1559929, 1560929), (1560929, 1561929), (1561...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,35950496,20221214_125409_om_and_comp_6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,32998593,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,34807,1.0,6_1_top_1_base_2_vs_6_3,win_non_comp,20221214_125409_om_and_comp_6_1_and_6_3.1,...,"[1.6555785442563817, 1.6866235492876445, 1.664...","[0.6307901161394641, 0.6655381193431421, 0.701...","[0.4673908122355257, 0.38359419544996237, 0.69...","[0.900289992310536, 0.2807581521168483, 0.1320...","[(32998593, 33018593), (33018593, 33038593), (...","[(32798593, 32818593), (32818593, 32838593), (...","[(1649929, 1650929), (1650929, 1651929), (1651...","[(1639929, 1640929), (1640929, 1641929), (1641...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,37450494,20221214_125409_om_and_comp_6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,34498591,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,36677,1.0,6_1_top_1_base_2_vs_6_3,win_non_comp,20221214_125409_om_and_comp_6_1_and_6_3.1,...,"[0.9562352171082348, 0.8615465705681848, 0.756...","[0.15955796441347841, 0.18843088608181321, 0.2...","[0.23449342689004346, 0.25460671878631813, 0.2...","[0.3903027043299651, 0.8705322845545354, 0.756...","[(34498591, 34518591), (34518591, 34538591), (...","[(34298591, 34318591), (34318591, 34338591), (...","[(1724929, 1725929), (1725929, 1726929), (1726...","[(1714929, 1715929)

In [34]:
merged_trials_and_video.columns

Index(['time', 'recording_dir', 'recording_file', 'time_stamp_index',
       'video_file', 'video_frame', 'video_number', 'subject_info',
       'competition_closeness', 'video_name', 'all_subjects',
       'current_subject', 'trial_outcome', 'lfp_index',
       'baseline_lfp_timestamp_range', 'trial_lfp_timestamp_range',
       'baseline_ephys_timestamp_range', 'trial_ephys_timestamp_range',
       'baseline_videoframe_range', 'trial_videoframe_range', 'trial_number',
       'Cohort', 'spike_interface_mPFC', 'spike_interface_vHPC',
       'spike_interface_BLA', 'spike_interface_LH', 'spike_interface_MD',
       'file_path', 'start_frame', 'stop_frame', 'individual_subj', 'all_subj',
       'recording_name', 'track_names', 'subject_id', 'corner_path',
       'corner_parts', 'bottom_width', 'top_width', 'right_height',
       'left_height', 'average_height', 'average_width', 'width_ratio',
       'height_ratio', 'rescaled_locations', 'box_bottom_left', 'reward_port',
       'box_top_lef

### Getting the subject IDs from the file name

In [20]:
TONE_TIMESTAMP_DF = pd.read_pickle("./proc/melted_trials_and_video.pkl")


In [21]:
all_trials_df = TONE_TIMESTAMP_DF.dropna(subset="condition").reset_index(drop=True)

In [22]:
all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,trial_or_baseline,current_thorax_velocity,current_chunked_thorax_velocity,chunked_lfp_index,chunked_ephys_timestamp,binned_current_thorax_velocity
0,30950500.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,27998597.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,28574,1.0,6_1_top_1_base_2_vs_6_3,...,"[(27998597.0, 28018597.0), (28018597.0, 280385...","[(27798597.0, 27818597.0), (27818597.0, 278385...","[(1399929.0, 1400929.0), (1400929.0, 1401929.0...","[(1389929.0, 1390929.0), (1390929.0, 1391929.0...",baseline,"[2.036892327741516, 2.241836602029708, 2.57597...","[2.3533739428089473, 1.434358857024035, 3.0417...","[(1389929.0, 1390929.0), (1390929.0, 1391929.0...","[(27798597.0, 27818597.0), (27818597.0, 278385...","[0, 0, 0, 0, 2, 0, 1, 1, 0, 0]"
1,30950500.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,27998597.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,28574,1.0,6_1_top_1_base_2_vs_6_3,...,"[(27998597.0, 28018597.0), (28018597.0, 280385...","[(27798597.0, 27818597.0), (27818597.0, 278385...","[(1399929.0, 1400929.0), (1400929.0, 1401929.0...","[(1389929.0, 1390929.0), (1390929.0, 1391929.0...",trial,"[0.7121153530651193, 0.6970557589759765, 0.724...","[0.9495329201051134, 3.0316038473645275, 1.981...","[(1399929.0, 1400929.0), (1400929.0, 1401929.0...","[(27998597.0, 28018597.0), (28018597.0, 280385...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
2,32550496.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,29598593.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,30569,1.0,6_1_top_1_base_2_vs_6_3,...,"[(29598593.0, 29618593.0), (29618593.0, 296385...","[(29398593.0, 29418593.0), (29418593.0, 294385...","[(1479929.0, 1480929.0), (1480929.0, 1481929.0...","[(1469929.0, 1470929.0), (1470929.0, 1471929.0...",baseline,"[2.9530693043854934, 2.5889456564677213, 2.295...","[0.9727462488501508, 0.14160626330067091, 0.15...","[(1469929.0, 1470929.0), (1470929.0, 1471929.0...","[(29398593.0, 29418593.0), (29418593.0, 294385...","[0, 0, 0, 0, 0, 2, 1, 1, 0, 0]"
3,32550496.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,29598593.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,30569,1.0,6_1_top_1_base_2_vs_6_3,...,"[(29598593.0, 29618593.0), (29618593.0, 296385...","[(29398593.0, 29418593.0), (29418593.0, 294385...","[(1479929.0, 1480929.0), (1480929.0, 1481929.0...","[(1469929.0, 1470929.0), (1470929.0, 1471929.0...",trial,"[1.1091618874765725, 1.2067853545334786, 1.308...","[1.0503547684805281, 0.8329540254506201, 1.119...","[(1479929.0, 1480929.0), (1480929.0, 1481929.0...","[(29598593.0, 29618593.0), (29618593.0, 296385...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
4,34350494.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,31398591.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,32813,1.0,6_1_top_1_base_2_vs_6_3,...,"[(31398591.0, 31418591.0), (31418591.0, 314385...","[(31198591.0, 31218591.0), (31218591.0, 312385...","[(1569929.0, 1570929.0), (1570929.0, 1571929.0...","[(1559929.0, 1560929.0), (1560929.0, 1561929.0...",baseline,"[5.081365648592532, 4.988102332136219, 4.87467...","[3.3220810860024765, 0.6014839466006836, 0.252...","[(1559929.0, 1560929.0), (1560929.0, 1561929.0...","[(31198591.0, 31218591.0), (31218591.0, 312385...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


- Original timestamps are based on ephys recordings at 20kHz. The LFP will be at 1kHz, so we will need to divide all the timestamps by 20

In [23]:
all_trials_df["resampled_index"] = all_trials_df["time_stamp_index"] // 20

In [24]:
all_trials_df["recording_dir"].unique()

array(['subj6_1_and_6_3',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1',
       '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2'],
      dtype=object)

- Getting a list of all the subjects through the recording name

In [25]:
all_trials_df["all_subjects"] = all_trials_df["recording_dir"].apply(lambda x: ["{}.{}".format(tup[0],tup[1]) for tup in re.findall(r'(\d+)-(\d+)', x.replace("_", "-"))[1:]])

In [26]:
all_trials_df["all_subjects"].head()

0    [6.3]
1    [6.3]
2    [6.3]
3    [6.3]
4    [6.3]
Name: all_subjects, dtype: object

- Getting the current subject of the recording through the ending of the recording name file

In [27]:
all_trials_df["subject_info"].head()

0    6_1_top_1_base_2_vs_6_3
1    6_1_top_1_base_2_vs_6_3
2    6_1_top_1_base_2_vs_6_3
3    6_1_top_1_base_2_vs_6_3
4    6_1_top_1_base_2_vs_6_3
Name: subject_info, dtype: object

In [28]:
all_trials_df["current_subject"] = all_trials_df["subject_info"].apply(lambda x: ".".join(x.replace("-","_").split("_")[:2]))

In [29]:
all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,trial_or_baseline,current_thorax_velocity,current_chunked_thorax_velocity,chunked_lfp_index,chunked_ephys_timestamp,binned_current_thorax_velocity
0,30950500.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,27998597.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,28574,1.0,6_1_top_1_base_2_vs_6_3,...,"[(27998597.0, 28018597.0), (28018597.0, 280385...","[(27798597.0, 27818597.0), (27818597.0, 278385...","[(1399929.0, 1400929.0), (1400929.0, 1401929.0...","[(1389929.0, 1390929.0), (1390929.0, 1391929.0...",baseline,"[2.036892327741516, 2.241836602029708, 2.57597...","[2.3533739428089473, 1.434358857024035, 3.0417...","[(1389929.0, 1390929.0), (1390929.0, 1391929.0...","[(27798597.0, 27818597.0), (27818597.0, 278385...","[0, 0, 0, 0, 2, 0, 1, 1, 0, 0]"
1,30950500.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,27998597.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,28574,1.0,6_1_top_1_base_2_vs_6_3,...,"[(27998597.0, 28018597.0), (28018597.0, 280385...","[(27798597.0, 27818597.0), (27818597.0, 278385...","[(1399929.0, 1400929.0), (1400929.0, 1401929.0...","[(1389929.0, 1390929.0), (1390929.0, 1391929.0...",trial,"[0.7121153530651193, 0.6970557589759765, 0.724...","[0.9495329201051134, 3.0316038473645275, 1.981...","[(1399929.0, 1400929.0), (1400929.0, 1401929.0...","[(27998597.0, 28018597.0), (28018597.0, 280385...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
2,32550496.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,29598593.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,30569,1.0,6_1_top_1_base_2_vs_6_3,...,"[(29598593.0, 29618593.0), (29618593.0, 296385...","[(29398593.0, 29418593.0), (29418593.0, 294385...","[(1479929.0, 1480929.0), (1480929.0, 1481929.0...","[(1469929.0, 1470929.0), (1470929.0, 1471929.0...",baseline,"[2.9530693043854934, 2.5889456564677213, 2.295...","[0.9727462488501508, 0.14160626330067091, 0.15...","[(1469929.0, 1470929.0), (1470929.0, 1471929.0...","[(29398593.0, 29418593.0), (29418593.0, 294385...","[0, 0, 0, 0, 0, 2, 1, 1, 0, 0]"
3,32550496.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,29598593.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,30569,1.0,6_1_top_1_base_2_vs_6_3,...,"[(29598593.0, 29618593.0), (29618593.0, 296385...","[(29398593.0, 29418593.0), (29418593.0, 294385...","[(1479929.0, 1480929.0), (1480929.0, 1481929.0...","[(1469929.0, 1470929.0), (1470929.0, 1471929.0...",trial,"[1.1091618874765725, 1.2067853545334786, 1.308...","[1.0503547684805281, 0.8329540254506201, 1.119...","[(1479929.0, 1480929.0), (1480929.0, 1481929.0...","[(29598593.0, 29618593.0), (29618593.0, 296385...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
4,34350494.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,31398591.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,32813,1.0,6_1_top_1_base_2_vs_6_3,...,"[(31398591.0, 31418591.0), (31418591.0, 314385...","[(31198591.0, 31218591.0), (31218591.0, 312385...","[(1569929.0, 1570929.0), (1570929.0, 1571929.0...","[(1559929.0, 1560929.0), (1560929.0, 1561929.0...",baseline,"[5.081365648592532, 4.988102332136219, 4.87467...","[3.3220810860024765, 0.6014839466006836, 0.252...","[(1559929.0, 1560929.0), (1560929.0, 1561929.0...","[(31198591.0, 31218591.0), (31218591.0, 312385...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


- Labeling the trial as a winner or loser if the winner matches the subject id or not

In [30]:
all_trials_df["trial_outcome"] = all_trials_df.apply(
    lambda x: "win" if str(x["condition"]).strip() == str(x["current_subject"]) 
             else ("lose" if str(x["condition"]) in x["all_subjects"] 
                   else x["condition"]), axis=1)

In [31]:
all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,trial_or_baseline,current_thorax_velocity,current_chunked_thorax_velocity,chunked_lfp_index,chunked_ephys_timestamp,binned_current_thorax_velocity
0,30950500.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,27998597.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,28574,1.0,6_1_top_1_base_2_vs_6_3,...,"[(27998597.0, 28018597.0), (28018597.0, 280385...","[(27798597.0, 27818597.0), (27818597.0, 278385...","[(1399929.0, 1400929.0), (1400929.0, 1401929.0...","[(1389929.0, 1390929.0), (1390929.0, 1391929.0...",baseline,"[2.036892327741516, 2.241836602029708, 2.57597...","[2.3533739428089473, 1.434358857024035, 3.0417...","[(1389929.0, 1390929.0), (1390929.0, 1391929.0...","[(27798597.0, 27818597.0), (27818597.0, 278385...","[0, 0, 0, 0, 2, 0, 1, 1, 0, 0]"
1,30950500.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,27998597.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,28574,1.0,6_1_top_1_base_2_vs_6_3,...,"[(27998597.0, 28018597.0), (28018597.0, 280385...","[(27798597.0, 27818597.0), (27818597.0, 278385...","[(1399929.0, 1400929.0), (1400929.0, 1401929.0...","[(1389929.0, 1390929.0), (1390929.0, 1391929.0...",trial,"[0.7121153530651193, 0.6970557589759765, 0.724...","[0.9495329201051134, 3.0316038473645275, 1.981...","[(1399929.0, 1400929.0), (1400929.0, 1401929.0...","[(27998597.0, 28018597.0), (28018597.0, 280385...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
2,32550496.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,29598593.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,30569,1.0,6_1_top_1_base_2_vs_6_3,...,"[(29598593.0, 29618593.0), (29618593.0, 296385...","[(29398593.0, 29418593.0), (29418593.0, 294385...","[(1479929.0, 1480929.0), (1480929.0, 1481929.0...","[(1469929.0, 1470929.0), (1470929.0, 1471929.0...",baseline,"[2.9530693043854934, 2.5889456564677213, 2.295...","[0.9727462488501508, 0.14160626330067091, 0.15...","[(1469929.0, 1470929.0), (1470929.0, 1471929.0...","[(29398593.0, 29418593.0), (29418593.0, 294385...","[0, 0, 0, 0, 0, 2, 1, 1, 0, 0]"
3,32550496.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,29598593.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,30569,1.0,6_1_top_1_base_2_vs_6_3,...,"[(29598593.0, 29618593.0), (29618593.0, 296385...","[(29398593.0, 29418593.0), (29418593.0, 294385...","[(1479929.0, 1480929.0), (1480929.0, 1481929.0...","[(1469929.0, 1470929.0), (1470929.0, 1471929.0...",trial,"[1.1091618874765725, 1.2067853545334786, 1.308...","[1.0503547684805281, 0.8329540254506201, 1.119...","[(1479929.0, 1480929.0), (1480929.0, 1481929.0...","[(29598593.0, 29618593.0), (29618593.0, 296385...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
4,34350494.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,31398591.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,32813,1.0,6_1_top_1_base_2_vs_6_3,...,"[(31398591.0, 31418591.0), (31418591.0, 314385...","[(31198591.0, 31218591.0), (31218591.0, 312385...","[(1569929.0, 1570929.0), (1570929.0, 1571929.0...","[(1559929.0, 1560929.0), (1560929.0, 1561929.0...",baseline,"[5.081365648592532, 4.988102332136219, 4.87467...","[3.3220810860024765, 0.6014839466006836, 0.252...","[(1559929.0, 1560929.0), (1560929.0, 1561929.0...","[(31198591.0, 31218591.0), (31218591.0, 312385...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [32]:
competition_closeness_map = {k: "non_comp" if "only" in str(k).lower() else "comp" if type(k) is str else np.nan for k in all_trials_df["competition_closeness"].unique()}

In [33]:
competition_closeness_map

{'Subj 1 Only': 'non_comp',
 'Subj 2 Only': 'non_comp',
 'Subj 2 blocking Subj 1': 'comp',
 'Subj 1 blocking Subj 2': 'comp',
 'Subj 1 then Subj 2': 'comp',
 'Subj 2 then Subj 1': 'comp',
 'Close Call': 'comp',
 nan: nan}

In [34]:
all_trials_df["competition_closeness"] = all_trials_df["competition_closeness"].map(competition_closeness_map)

In [35]:
all_trials_df["competition_closeness"]

0      non_comp
1      non_comp
2      non_comp
3      non_comp
4      non_comp
         ...   
551         NaN
552         NaN
553         NaN
554         NaN
555         NaN
Name: competition_closeness, Length: 556, dtype: object

In [36]:
all_trials_df["competition_closeness"] = all_trials_df.apply(lambda x: "_".join([str(x["trial_outcome"]), str(x["competition_closeness"])]).strip("nan").strip("_"), axis=1)

### Extracting the LFP

In [37]:
all_trials_df

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,trial_or_baseline,current_thorax_velocity,current_chunked_thorax_velocity,chunked_lfp_index,chunked_ephys_timestamp,binned_current_thorax_velocity
0,30950500.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,27998597.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,28574,1.0,6_1_top_1_base_2_vs_6_3,...,"[(27998597.0, 28018597.0), (28018597.0, 280385...","[(27798597.0, 27818597.0), (27818597.0, 278385...","[(1399929.0, 1400929.0), (1400929.0, 1401929.0...","[(1389929.0, 1390929.0), (1390929.0, 1391929.0...",baseline,"[2.036892327741516, 2.241836602029708, 2.57597...","[2.3533739428089473, 1.434358857024035, 3.0417...","[(1389929.0, 1390929.0), (1390929.0, 1391929.0...","[(27798597.0, 27818597.0), (27818597.0, 278385...","[0, 0, 0, 0, 2, 0, 1, 1, 0, 0]"
1,30950500.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,27998597.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,28574,1.0,6_1_top_1_base_2_vs_6_3,...,"[(27998597.0, 28018597.0), (28018597.0, 280385...","[(27798597.0, 27818597.0), (27818597.0, 278385...","[(1399929.0, 1400929.0), (1400929.0, 1401929.0...","[(1389929.0, 1390929.0), (1390929.0, 1391929.0...",trial,"[0.7121153530651193, 0.6970557589759765, 0.724...","[0.9495329201051134, 3.0316038473645275, 1.981...","[(1399929.0, 1400929.0), (1400929.0, 1401929.0...","[(27998597.0, 28018597.0), (28018597.0, 280385...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
2,32550496.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,29598593.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,30569,1.0,6_1_top_1_base_2_vs_6_3,...,"[(29598593.0, 29618593.0), (29618593.0, 296385...","[(29398593.0, 29418593.0), (29418593.0, 294385...","[(1479929.0, 1480929.0), (1480929.0, 1481929.0...","[(1469929.0, 1470929.0), (1470929.0, 1471929.0...",baseline,"[2.9530693043854934, 2.5889456564677213, 2.295...","[0.9727462488501508, 0.14160626330067091, 0.15...","[(1469929.0, 1470929.0), (1470929.0, 1471929.0...","[(29398593.0, 29418593.0), (29418593.0, 294385...","[0, 0, 0, 0, 0, 2, 1, 1, 0, 0]"
3,32550496.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,29598593.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,30569,1.0,6_1_top_1_base_2_vs_6_3,...,"[(29598593.0, 29618593.0), (29618593.0, 296385...","[(29398593.0, 29418593.0), (29418593.0, 294385...","[(1479929.0, 1480929.0), (1480929.0, 1481929.0...","[(1469929.0, 1470929.0), (1470929.0, 1471929.0...",trial,"[1.1091618874765725, 1.2067853545334786, 1.308...","[1.0503547684805281, 0.8329540254506201, 1.119...","[(1479929.0, 1480929.0), (1480929.0, 1481929.0...","[(29598593.0, 29618593.0), (29618593.0, 296385...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
4,34350494.0,1.0,subj6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,dio_ECU_Din1,31398591.0,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,32813,1.0,6_1_top_1_base_2_vs_6_3,...,"[(31398591.0, 31418591.0), (31418591.0, 314385...","[(31198591.0, 31218591.0), (31218591.0, 312385...","[(1569929.0, 1570929.0), (1570929.0, 1571929.0...","[(1559929.0, 1560929.0), (1560929.0, 1561929.0...",baseline,"[5.081365648592532, 4.988102332136219, 4.87467...","[3.3220810860024765, 0.6014839466006836, 0.252...","[(1559929.0, 1560929.0), (1560929.0, 1561929.0...","[(31198591.0, 31218591.0), (31218591.0, 312385...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,68781880.0,1.0,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,dio_ECU_Din1,65588836.0,20230621_111240_standard_comp_to_omission_D5_s...,65550,1.0,1-4_t3b3L_box1,...,"[(65588836.0, 65608836.0), (65608836.0, 656288...","[(

In [38]:
all_trials_df["recording_dir"].unique()

array(['subj6_1_and_6_3',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1',
       '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2'],
      dtype=object)

In [ ]:
recording_name_to_all_ch_lfp = {}
# Going through all the recording sessions 
for session_dir in ALL_SESSION_DIR:
    # Going through all the recordings in each session
    for recording_path in glob.glob(os.path.join(session_dir, "*.rec")):
        try:
            recording_basename = os.path.splitext(os.path.basename(recording_path))[0]
            # checking to see if the recording has an ECU component
            # if it doesn't, then the next one be extracted
            current_recording = se.read_spikegadgets(recording_path, stream_id="ECU")
            current_recording = se.read_spikegadgets(recording_path, stream_id="trodes")
            print(recording_basename)
            # Preprocessing the LFP
            current_recording = sp.bandpass_filter(current_recording, freq_min=0.5, freq_max=300)
            current_recording = sp.notch_filter(current_recording, freq=60)
            current_recording = sp.resample(current_recording, resample_rate=1000)
            current_recording = sp.zscore(current_recording)
            recording_name_to_all_ch_lfp[recording_basename] = current_recording
        except:
            pass



20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged
20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged
20221203_154800_omission_and_competition_subject_6_1_top_1_base_3_merged
20221202_134600_omission_and_competition_subject_6_1_top_2_base_3_merged
20221214_125409_om_and_comp_6_1_top_1_base_2_vs_6_3
20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged
20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged
20230619_115321_standard_comp_to_omission_D3_subj_1-4_t3b3L_box2_merged
20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged
20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged
20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged
20221215_145401_comp_amd_om_6_1_top_4_base_3


- Filtering for all trials that have labels

In [ ]:
all_trials_df = all_trials_df[all_trials_df["recording_file"].isin(recording_name_to_all_ch_lfp.keys())].reset_index(drop=True)

In [ ]:
all_trials_df.head()

In [ ]:
all_trials_df["trial_outcome"].unique()

In [ ]:
all_trials_df["trial_or_baseline"].unique()

In [ ]:
recording_name_to_all_ch_lfp.keys()

In [ ]:
CHANNEL_MAPPING_DF

## Power Calculation

In [ ]:
all_trials_df.head()

- Adding all the brain region to ch information

In [ ]:
channel_map_and_all_trials_df = all_trials_df.merge(CHANNEL_MAPPING_DF, left_on="current_subject", right_on="Subject", how="left")

- Linking up all LFP calculations with all the trials

In [ ]:
channel_map_and_all_trials_df["all_ch_lfp"] = channel_map_and_all_trials_df["recording_file"].map(recording_name_to_all_ch_lfp)

- Creating a new row for each brain region

In [ ]:
brain_region_col = [col for col in CHANNEL_MAPPING_DF if "spike_interface" in col]

In [ ]:
id_cols = [col for col in channel_map_and_all_trials_df.columns if col not in brain_region_col]

In [ ]:
melted_channel_map_and_all_trials_df = channel_map_and_all_trials_df.melt(id_vars=id_cols ,value_vars=brain_region_col, var_name='brain_region', value_name='channel')


In [ ]:
melted_channel_map_and_all_trials_df.head()

In [ ]:
melted_channel_map_and_all_trials_df.tail()

- Getting the traces for each trial for each brain region

In [ ]:
melted_channel_map_and_all_trials_df["channel"] = melted_channel_map_and_all_trials_df["channel"].astype(int).astype(str)
melted_channel_map_and_all_trials_df["resampled_index"] = melted_channel_map_and_all_trials_df["resampled_index"].astype(int)

In [ ]:
melted_channel_map_and_all_trials_df["chunked_lfp_index"].iloc[0]

In [ ]:
melted_channel_map_and_all_trials_df["binned_current_thorax_velocity"].iloc[0]

In [ ]:
melted_channel_map_and_all_trials_df["chunked_current_trace"] = melted_channel_map_and_all_trials_df.apply(lambda x: 
[x["all_ch_lfp"].get_traces(channel_ids=[x["channel"]], start_frame=chunk_index[0], end_frame=chunk_index[1]).T[0] for chunk_index in x["chunked_lfp_index"]], axis=1)

In [ ]:
len(melted_channel_map_and_all_trials_df["chunked_current_trace"].iloc[0])

In [ ]:
melted_channel_map_and_all_trials_df["chunked_current_trace"]

In [ ]:
melted_channel_map_and_all_trials_df["chunk_id"] = [list(range(0,10))] * len(melted_channel_map_and_all_trials_df)

In [ ]:
exploded_channel_map_and_all_trials_df = melted_channel_map_and_all_trials_df.explode(["current_chunked_thorax_velocity", "chunked_current_trace", "binned_current_thorax_velocity", "chunked_lfp_index",	"chunked_ephys_timestamp", "chunk_id"])

In [ ]:
exploded_channel_map_and_all_trials_df

In [ ]:
exploded_channel_map_and_all_trials_df["len_chunked_current_trace"] = exploded_channel_map_and_all_trials_df["chunked_current_trace"].apply(lambda x: len(x))


In [ ]:
exploded_channel_map_and_all_trials_df["len_chunked_current_trace"].unique()

In [ ]:
exploded_channel_map_and_all_trials_df["chunked_current_trace"] = exploded_channel_map_and_all_trials_df["chunked_current_trace"].apply(lambda x: x[:RESAMPLE_RATE])

In [ ]:
exploded_channel_map_and_all_trials_df["chunked_current_trace"]

- Calcuating the power at each frequency band

In [ ]:
exploded_channel_map_and_all_trials_df["multitaper"] = exploded_channel_map_and_all_trials_df["chunked_current_trace"].apply(lambda x: Multitaper(time_series=x,                                                                                        sampling_frequency=RESAMPLE_RATE))

In [ ]:
exploded_channel_map_and_all_trials_df["connectivity"] = exploded_channel_map_and_all_trials_df["multitaper"].apply(lambda x: Connectivity.from_multitaper(x))

In [ ]:
exploded_channel_map_and_all_trials_df["frequencies"] = exploded_channel_map_and_all_trials_df["connectivity"].apply(lambda x: x.frequencies)

In [ ]:
exploded_channel_map_and_all_trials_df["power"] = exploded_channel_map_and_all_trials_df["connectivity"].apply(lambda x: x.power().squeeze())

In [ ]:
exploded_channel_map_and_all_trials_df.head()

In [ ]:
exploded_channel_map_and_all_trials_df.tail()

# Plotting by velocity

- Grouping all the rows(1s segment of trials) that share the same brain region, whether its before or after the tone, and velocity grouping

In [ ]:
grouped_all_trials_df = exploded_channel_map_and_all_trials_df.groupby(['brain_region', 'trial_or_baseline', 'binned_current_thorax_velocity']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

- Calculating the mean and sem of the LFP power

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]

    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.01, 0.1)

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} for {}".format(BIN_TO_VELOCITY[row["binned_current_thorax_velocity"]], \
            row["trial_or_baseline"]), color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]],
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)

            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], \
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2,
            color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]])
        except Exception as e: 
            print(e)
            continue

    plt.legend(loc="lower left")
    
    plt.savefig("./proc/velocity_parsing/plot_region_line_velocity_baselinetrial/\
    rce_lfp_power_velocity_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))


In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]

    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} for {}".format(BIN_TO_VELOCITY[row["binned_current_thorax_velocity"]], \
            row["trial_or_baseline"]), color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]],
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)

            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], \
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2,
            color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]])
        except Exception as e: 
            print(e)
            continue

    plt.savefig("./proc/velocity_parsing/plot_region_line_velocity_baselinetrial/\
    rce_lfp_power_velocity_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))


In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]

    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    # plt.ylim(0, 0.005)
    plt.yscale("log")
    plt.ylim(0.0003, 0.005) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} for {}".format(BIN_TO_VELOCITY[row["binned_current_thorax_velocity"]], \
            row["trial_or_baseline"]), color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]],
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)

            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], \
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2,
            color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]])
        except Exception as e: 
            print(e)
            continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_velocity_baselinetrial/\
    rce_lfp_power_velocity_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))


## Plotting all the lower velocities together

In [ ]:
speed_filtered_all_trials_df = exploded_channel_map_and_all_trials_df[exploded_channel_map_and_all_trials_df["binned_current_thorax_velocity"] == 0]

In [ ]:
speed_filtered_all_trials_df.shape

In [ ]:
grouped_all_trials_df = speed_filtered_all_trials_df.groupby(['brain_region', 'trial_outcome', 'trial_or_baseline']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
grouped_all_trials_df["trial_outcome"].unique()

# Drawing plots all together

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq) 
    # plt.yscale("log")
    # plt.ylim(0.01, 0.06)

    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.1, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend(loc="lower left")
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.1, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend(loc="upper right")
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.0003, 0.005) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend(loc="upper right")
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

## Plotting all the higher velocities together

In [ ]:
speed_filtered_all_trials_df = exploded_channel_map_and_all_trials_df[exploded_channel_map_and_all_trials_df["binned_current_thorax_velocity"] == 1]

In [ ]:
exploded_channel_map_and_all_trials_df["theta_power"] = exploded_channel_map_and_all_trials_df["power"].apply(lambda x: x[4:13])

In [ ]:
exploded_channel_map_and_all_trials_df["theta_power"].iloc[0]

In [ ]:
exploded_channel_map_and_all_trials_df["averaged_theta_power"] = exploded_channel_map_and_all_trials_df["theta_power"].apply(lambda x: np.mean(x))

In [ ]:
exploded_channel_map_and_all_trials_df["averaged_theta_power"]

In [ ]:
exploded_channel_map_and_all_trials_df.columns

In [ ]:
exploded_channel_map_and_all_trials_df["all_thorax_velocity"].iloc[0].shape

In [ ]:
exploded_channel_map_and_all_trials_df.info()

In [ ]:
exploded_channel_map_and_all_trials_df.drop(columns=["multitaper", "connectivity", 'all_thorax_locations', 'all_thorax_velocity', 'all_ch_lfp']).to_pickle("./proc/rce_1_sec_interval_lfp_power_spectra_and_velocity.pkl")

In [ ]:
grouped_all_trials_df = speed_filtered_all_trials_df.groupby(['brain_region', 'trial_outcome', 'trial_or_baseline']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
exploded_channel_map_and_all_trials_df["frequencies"].iloc[0][4:13]

In [ ]:
grouped_all_trials_df["power"].iloc[0].shape

In [ ]:
grouped_all_trials_df["power"].iloc

# Drawing plots all together

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)   
    plt.yscale("log")
    plt.ylim(0.01, 0.1)

    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_higher_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_higher_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.0003, 0.008) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_higher_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

# OLD CODE BELOW

In [31]:
raise ValueError()

ValueError: 

In [30]:
channel_map_and_all_trials_df.columns

Index(['time', 'recording_dir', 'recording_file', 'time_stamp_index',
       'video_file', 'video_frame', 'video_number', 'subject_info',
       'competition_closeness', 'video_name', 'all_subjects',
       'current_subject', 'trial_outcome', 'lfp_index',
       'baseline_lfp_timestamp_range', 'trial_lfp_timestamp_range',
       'baseline_ephys_timestamp_range', 'trial_ephys_timestamp_range',
       'baseline_videoframe_range', 'trial_videoframe_range', 'trial_number',
       'Cohort', 'spike_interface_mPFC', 'spike_interface_vHPC',
       'spike_interface_BLA', 'spike_interface_LH', 'spike_interface_MD',
       'mPFC_baseline_lfp_trace', 'mPFC_trial_lfp_trace',
       'vHPC_baseline_lfp_trace', 'vHPC_trial_lfp_trace',
       'BLA_baseline_lfp_trace', 'BLA_trial_lfp_trace',
       'LH_baseline_lfp_trace', 'LH_trial_lfp_trace', 'MD_baseline_lfp_trace',
       'MD_trial_lfp_trace'],
      dtype='object')

In [29]:
merged_trials_and_video.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,video_name,...,baseline_thorax_velocity,trial_thorax_velocity,trial_chunked_thorax_velocity,baseline_chunked_thorax_velocity,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,binned_baseline_thorax_velocity,binned_trial_thorax_velocity
0,30950500,20221214_125409_om_and_comp_6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,27998597,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,28574,1.0,6_1_top_1_base_2_vs_6_3,win_non_comp,20221214_125409_om_and_comp_6_1_and_6_3.1,...,"[1.2476140081638165, 1.1366289071218907, 1.068...","[0.18170472850681613, 0.16961378983662276, 0.2...","[0.49872187702582166, 1.0783705041183183, 1.43...","[1.6838472335457935, 1.3726890573782602, 1.887...","[(27998597, 28018597), (28018597, 28038597), (...","[(27798597, 27818597), (27818597, 27838597), (...","[(1399929, 1400929), (1400929, 1401929), (1401...","[(1389929, 1390929), (1390929, 1391929), (1391...","[0, 0, 0, 0, 1, 1, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,32550496,20221214_125409_om_and_comp_6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,29598593,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,30569,1.0,6_1_top_1_base_2_vs_6_3,win_non_comp,20221214_125409_om_and_comp_6_1_and_6_3.1,...,"[2.7236727895073223, 2.628037603947678, 2.5298...","[0.38577700470932025, 0.16352840888997497, 0.0...","[0.3129891797005337, 0.210547774617734, 0.2517...","[1.458421075097482, 0.23609874826191488, 0.073...","[(29598593, 29618593), (29618593, 29638593), (...","[(29398593, 29418593), (29418593, 29438593), (...","[(1479929, 1480929), (1480929, 1481929), (1481...","[(1469929, 1470929), (1470929, 1471929), (1471...","[0, 0, 0, 0, 0, 1, 1, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,34350494,20221214_125409_om_and_comp_6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,31398591,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,32813,1.0,6_1_top_1_base_2_vs_6_3,win_non_comp,20221214_125409_om_and_comp_6_1_and_6_3.1,...,"[5.210405290871598, 5.31421978904958, 5.379251...","[0.16560567036540477, 0.1278165569746918, 0.09...","[0.19868857983610794, 0.48118846206642074, 0.4...","[3.439626522203859, 1.1535882532998658, 0.0565...","[(31398591, 31418591), (31418591, 31438591), (...","[(31198591, 31218591), (31218591, 31238591), (...","[(1569929, 1570929), (1570929, 1571929), (1571...","[(1559929, 1560929), (1560929, 1561929), (1561...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,35950496,20221214_125409_om_and_comp_6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,32998593,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,34807,1.0,6_1_top_1_base_2_vs_6_3,win_non_comp,20221214_125409_om_and_comp_6_1_and_6_3.1,...,"[1.6555785442563817, 1.6866235492876445, 1.664...","[0.6307901161394641, 0.6655381193431421, 0.701...","[0.4673908122355257, 0.38359419544996237, 0.69...","[0.900289992310536, 0.2807581521168483, 0.1320...","[(32998593, 33018593), (33018593, 33038593), (...","[(32798593, 32818593), (32818593, 32838593), (...","[(1649929, 1650929), (1650929, 1651929), (1651...","[(1639929, 1640929), (1640929, 1641929), (1641...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,37450494,20221214_125409_om_and_comp_6_1_and_6_3,20221214_125409_om_and_comp_6_1_top_1_base_2_v...,34498591,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,36677,1.0,6_1_top_1_base_2_vs_6_3,win_non_comp,20221214_125409_om_and_comp_6_1_and_6_3.1,...,"[0.9562352171082348, 0.8615465705681848, 0.756...","[0.15955796441347841, 0.18843088608181321, 0.2...","[0.23449342689004346, 0.25460671878631813, 0.2...","[0.3903027043299651, 0.8705322845545354, 0.756...","[(34498591, 34518591), (34518591, 34538591), (...","[(34298591, 34318591), (34318591, 34338591), (...","[(1724929, 1725929), (1725929, 1726929), (1726...","[(1714929, 1715929)

In [ ]:
pd.merge(left=channel_map_and_all_trials_df, right=merged_trials_and_video, left_on=)

In [11]:
trace_columns = [col for col in channel_map_and_all_trials_df.columns if "trace" in col]

In [12]:
id_cols = [col for col in channel_map_and_all_trials_df.columns if col not in trace_columns]

In [13]:
trace_columns

['mPFC_baseline_lfp_trace',
 'mPFC_trial_lfp_trace',
 'vHPC_baseline_lfp_trace',
 'vHPC_trial_lfp_trace',
 'BLA_baseline_lfp_trace',
 'BLA_trial_lfp_trace',
 'LH_baseline_lfp_trace',
 'LH_trial_lfp_trace',
 'MD_baseline_lfp_trace',
 'MD_trial_lfp_trace']

In [14]:
channel_map_and_all_trials_df = channel_map_and_all_trials_df.reset_index()

In [15]:
melted_channel_map_and_all_trials_df = channel_map_and_all_trials_df.melt(id_vars=id_cols ,value_vars=trace_columns, var_name='region_and_baseline_or_trial', value_name='lfp_trace')

In [16]:
melted_channel_map_and_all_trials_df["baseline_or_trial"] = melted_channel_map_and_all_trials_df["region_and_baseline_or_trial"].apply(lambda x: x.split("_")[1])

melted_channel_map_and_all_trials_df["brain_region"] = melted_channel_map_and_all_trials_df["region_and_baseline_or_trial"].apply(lambda x: x.split("_")[0])

In [17]:
melted_channel_map_and_all_trials_df.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,video_name,...,Cohort,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD,region_and_baseline_or_trial,lfp_trace,baseline_or_trial,brain_region
0,6310663,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,1390826,20221202_134600_omission_and_competition_subje...,1734,1.0,6_1_top_2_base_3,rewarded,20221202_134600_omission_and_competition_subje...,...,1,21,15,14,13,16,mPFC_baseline_lfp_trace,"[1.8457601, 1.7363818, 1.6475118, 1.59738, 1.2...",baseline,mPFC
1,7910662,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,2990825,20221202_134600_omission_and_competition_subje...,3728,1.0,6_1_top_2_base_3,rewarded,20221202_134600_omission_and_competition_subje...,...,1,21,15,14,13,16,mPFC_baseline_lfp_trace,"[1.2191132, 1.1348007, 1.2054409, 1.0960625, 0...",baseline,mPFC
2,9710660,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,4790823,20221202_134600_omission_and_competition_subje...,5972,1.0,6_1_top_2_base_3,rewarded,20221202_134600_omission_and_competition_subje...,...,1,21,15,14,13,16,mPFC_baseline_lfp_trace,"[-1.2669662, -1.2965895, -1.2532939, -0.986684...",baseline,mPFC
3,11310658,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,6390821,20221202_134600_omission_and_competition_subje...,7966,1.0,6_1_top_2_base_3,omission,20221202_134600_omission_and_competition_subje...,...,1,21,15,14,13,16,mPFC_baseline_lfp_trace,"[-2.0257788, -2.0348935, -1.9323514, -1.754611...",baseline,mPFC
4,12810657,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,7890820,20221202_134600_omission_and_competition_subje...,9836,1.0,6_1_top_2_base_3,rewarded,20221202_134600_omission_and_competition_subje...,...,1,21,15,14,13,16,mPFC_baseline_lfp_trace,"[-0.5765152, 0.25749493, 0.6403192, 0.4375135,...",baseline,mPFC


In [18]:
raise ValueError()

ValueError: 

# Calculating power

In [ ]:
melted_channel_map_and_all_trials_df["multitaper"] = melted_channel_map_and_all_trials_df["lfp_trace"].apply(lambda x: Multitaper(time_series=x,                                                                                        sampling_frequency=RESAMPLE_RATE,  time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT, time_window_duration=TIME_WINDOW_DURATION, time_window_step=TIME_WINDOW_STEP))

In [ ]:
melted_channel_map_and_all_trials_df["connectivity"] = melted_channel_map_and_all_trials_df["multitaper"].apply(lambda x: Connectivity.from_multitaper(x))

In [ ]:
melted_channel_map_and_all_trials_df["frequencies"] = melted_channel_map_and_all_trials_df["connectivity"].apply(lambda x: x.frequencies)

In [ ]:
melted_channel_map_and_all_trials_df["all_segment_power"] = melted_channel_map_and_all_trials_df["connectivity"].apply(lambda x: x.power().squeeze())

In [ ]:
melted_channel_map_and_all_trials_df["power"] = melted_channel_map_and_all_trials_df["all_segment_power"].apply(lambda x: np.mean(x, axis=0))

In [ ]:
melted_channel_map_and_all_trials_df.head()

In [ ]:
melted_channel_map_and_all_trials_df.to_pickle("./proc/rce_power_of_zscored_lfp.pkl")

# Plotting all the power's of each subject together

In [ ]:
grouping_column = "current_subject"

In [ ]:
grouped_all_trials_df = melted_channel_map_and_all_trials_df.groupby(['brain_region', 'baseline_or_trial', grouping_column]).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

# Drawing plots all together

In [ ]:
color_list = ["red", "blue", "green", "orange"]
list(grouped_all_trials_df["current_subject"].unique())

In [ ]:
subj_to_color = {subj: color_list[index] for index, subj in enumerate(list(grouped_all_trials_df["current_subject"].unique()))}

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq) 
    # plt.yscale("log")
    # plt.ylim(0.01, 0.06)

    for baseline_or_trial, outcome in itertools.product(region_df["baseline_or_trial"].unique(), sorted(region_df["current_subject"].unique())[::-1]):
        outcome_df = region_df[(region_df["baseline_or_trial"] == baseline_or_trial) & (region_df["current_subject"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=melted_channel_map_and_all_trials_df["frequencies"].iloc[0][low_freq:high_freq+1], y=row["mean_power"][low_freq:high_freq+1], \
                label="{} {}".format(outcome, baseline_or_trial), \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[baseline_or_trial], linewidth=3, color=subj_to_color[outcome])
                
                plt.fill_between(melted_channel_map_and_all_trials_df["frequencies"].iloc[0][low_freq:high_freq+1], 
                row["mean_power"][low_freq:high_freq+1] - row["sem_power"][low_freq:high_freq+1], row["mean_power"][low_freq:high_freq+1] + row["sem_power"][low_freq:high_freq+1], \
                alpha=0.1, color=subj_to_color[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/lfp_power/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}_outcome_{}.png".format(low_freq, high_freq, region.split("_")[-1], grouping_column))
    plt.show()

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq) 
    # plt.yscale("log")
    # plt.ylim(0.01, 0.06)

    for baseline_or_trial, outcome in itertools.product(region_df["baseline_or_trial"].unique(), sorted(region_df["current_subject"].unique())[::-1]):
        outcome_df = region_df[(region_df["baseline_or_trial"] == baseline_or_trial) & (region_df["current_subject"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=melted_channel_map_and_all_trials_df["frequencies"].iloc[0][low_freq:high_freq+1], y=row["mean_power"][low_freq:high_freq+1], \
                label="{} {}".format(outcome, baseline_or_trial), \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[baseline_or_trial], linewidth=3, color=subj_to_color[outcome])
                
                plt.fill_between(melted_channel_map_and_all_trials_df["frequencies"].iloc[0][low_freq:high_freq+1], 
                row["mean_power"][low_freq:high_freq+1] - row["sem_power"][low_freq:high_freq+1], row["mean_power"][low_freq:high_freq+1] + row["sem_power"][low_freq:high_freq+1], \
                alpha=0.1, color=subj_to_color[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/lfp_power/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}_outcome_{}.png".format(low_freq, high_freq, region.split("_")[-1], grouping_column))
    plt.show()

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq) 
    # plt.yscale("log")
    # plt.ylim(0.01, 0.06)

    for baseline_or_trial, outcome in itertools.product(region_df["baseline_or_trial"].unique(), sorted(region_df["current_subject"].unique())[::-1]):
        outcome_df = region_df[(region_df["baseline_or_trial"] == baseline_or_trial) & (region_df["current_subject"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=melted_channel_map_and_all_trials_df["frequencies"].iloc[0][low_freq:high_freq+1], y=row["mean_power"][low_freq:high_freq+1], \
                label="{} {}".format(outcome, baseline_or_trial), \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[baseline_or_trial], linewidth=3, color=subj_to_color[outcome])
                
                plt.fill_between(melted_channel_map_and_all_trials_df["frequencies"].iloc[0][low_freq:high_freq+1], 
                row["mean_power"][low_freq:high_freq+1] - row["sem_power"][low_freq:high_freq+1], row["mean_power"][low_freq:high_freq+1] + row["sem_power"][low_freq:high_freq+1], \
                alpha=0.1, color=subj_to_color[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/lfp_power/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}_outcome_{}.png".format(low_freq, high_freq, region.split("_")[-1], grouping_column))
    plt.show()

# Plotting all the power's of each trial outcome together

In [ ]:
grouping_column = "trial_outcome"

In [ ]:
grouped_all_trials_df = melted_channel_map_and_all_trials_df.groupby(['brain_region', 'baseline_or_trial', grouping_column]).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

# Drawing plots all together

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq) 
    # plt.yscale("log")
    # plt.ylim(0.01, 0.06)

    for baseline_or_trial, outcome in itertools.product(region_df["baseline_or_trial"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["baseline_or_trial"] == baseline_or_trial) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=melted_channel_map_and_all_trials_df["frequencies"].iloc[0][low_freq:high_freq+1], y=row["mean_power"][low_freq:high_freq+1], \
                label="{} {}".format(outcome, baseline_or_trial), \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[baseline_or_trial], linewidth=3, color=BASELINE_OUTCOME_TO_COLOR[outcome])
                
                plt.fill_between(melted_channel_map_and_all_trials_df["frequencies"].iloc[0][low_freq:high_freq+1], 
                row["mean_power"][low_freq:high_freq+1] - row["sem_power"][low_freq:high_freq+1], row["mean_power"][low_freq:high_freq+1] + row["sem_power"][low_freq:high_freq+1], \
                alpha=0.1, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/lfp_power/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}_outcome_{}.png".format(low_freq, high_freq, region.split("_")[-1], grouping_column))
    plt.show()

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq) 
    # plt.yscale("log")
    # plt.ylim(0.01, 0.06)

    for baseline_or_trial, outcome in itertools.product(region_df["baseline_or_trial"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["baseline_or_trial"] == baseline_or_trial) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=melted_channel_map_and_all_trials_df["frequencies"].iloc[0][low_freq:high_freq+1], y=row["mean_power"][low_freq:high_freq+1], \
                label="{} {}".format(outcome, baseline_or_trial), \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[baseline_or_trial], linewidth=3, color=BASELINE_OUTCOME_TO_COLOR[outcome])
                
                plt.fill_between(melted_channel_map_and_all_trials_df["frequencies"].iloc[0][low_freq:high_freq+1], 
                row["mean_power"][low_freq:high_freq+1] - row["sem_power"][low_freq:high_freq+1], row["mean_power"][low_freq:high_freq+1] + row["sem_power"][low_freq:high_freq+1], \
                alpha=0.1, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/lfp_power/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}_outcome_{}.png".format(low_freq, high_freq, region.split("_")[-1], grouping_column))
    plt.show()

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq) 
    # plt.yscale("log")
    # plt.ylim(0.01, 0.06)

    for baseline_or_trial, outcome in itertools.product(region_df["baseline_or_trial"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["baseline_or_trial"] == baseline_or_trial) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=melted_channel_map_and_all_trials_df["frequencies"].iloc[0][low_freq:high_freq+1], y=row["mean_power"][low_freq:high_freq+1], \
                label="{} {}".format(outcome, baseline_or_trial), \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[baseline_or_trial], linewidth=3, color=BASELINE_OUTCOME_TO_COLOR[outcome])
                
                plt.fill_between(melted_channel_map_and_all_trials_df["frequencies"].iloc[0][low_freq:high_freq+1], 
                row["mean_power"][low_freq:high_freq+1] - row["sem_power"][low_freq:high_freq+1], row["mean_power"][low_freq:high_freq+1] + row["sem_power"][low_freq:high_freq+1], \
                alpha=0.1, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/lfp_power/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}_outcome_{}.png".format(low_freq, high_freq, region.split("_")[-1], grouping_column))
    plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a4490980-3f6a-4f44-80eb-ebd789a5b21f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>